In [1]:
import os
os.listdir()

['.ipynb_checkpoints',
 'customers.json',
 'loan-2017-10.csv',
 'loan-2017-11.csv',
 'loan-2017-12.csv',
 'loan-2018-1.csv',
 'loan-2018-2.csv',
 'loan-2018-3.csv',
 'loan-2018-4.csv',
 'loan-2018-5.csv',
 'loan-2018-6.csv',
 'loan-2019-1.csv',
 'loan-2019-2.csv',
 'loan-2019-3.csv',
 'loan-2019-4.csv',
 'loan-2019-5.csv',
 'loan-2019-6.csv',
 'loan_files_concatenate.ipynb',
 'Untitled.ipynb',
 'visits.csv']

In [4]:
#Get all loan files data in one loan file
import os
import glob
import pandas
extension='csv'
all_loanfiles = [i for i in glob.glob('loan*.{}'.format(extension))]
#combine all files in the list
combined_csv = pandas.concat([pandas.read_csv(f) for f in all_loanfiles ])
#export to csv
combined_csv.to_csv( "All_Loans.csv", index=False, encoding='utf-8-sig')


In [31]:
import os
import glob
import pandas
import numpy
import json
import googletrans
from googletrans import Translator
translator = Translator()

GoodLoanFile=pandas.read_csv("All_Loans.csv")
## Remove unnamed column from file
GoodLoanFile=GoodLoanFile.drop("Unnamed: 0",axis=1)

##Remove special characters from WebVisit_id to get a proper Webvisit_id
GoodLoanFile['webvisit_id'] = GoodLoanFile['webvisit_id'].astype(str).str.extract('(\d+)')

## Merge loan and visits files

WebVisitFile= pandas.read_csv("visits.csv")

# Removed Unnamed and timestamp columns from visits file 
WebVisitFile=WebVisitFile.drop(["Unnamed: 0","timestamp"],axis=1)
WebVisitFile['id'] = WebVisitFile['id'].astype(str)
#All Loan data Left joined with Visits data
MergeVisit= pandas.merge(left=GoodLoanFile,right=WebVisitFile, how='left', left_on='webvisit_id', right_on='id')
MergeVisit=MergeVisit.drop(["id_y"],axis=1)
#Replaced NaN with an empty string for data to look better
MergeVisit=MergeVisit.replace(numpy.NaN, '', regex=True)
MergeVisit=MergeVisit.rename(columns={'id_x': 'loan_id'})

##Merge Customer File data with merged Loan and visit data i.e MergeVisit dataframe

CustomerData=pandas.read_json('http://rocker-data-engineering-task.storage.googleapis.com/data/customers.json',lines=True)

# Merge Customer data to MergeVisit dataframe to have all data together
CustomerLoanVisitData=pandas.merge(left=MergeVisit,right=CustomerData, how='left', left_on='user_id', right_on='id')
CustomerLoanVisitData=CustomerLoanVisitData.drop(["id"], axis=1)
#Rearranged Columns
CustomerLoanVisitData=CustomerLoanVisitData[["user_id","loan_id","webvisit_id","name","gender","birthday","ssn","city","zip_code","loan_amount","interest","loan_purpose","outcome","timestamp","referrer","campaign_name"]]
#sorted data on userid, by default it is sorted in ascending order
CustomerLoanVisitData=CustomerLoanVisitData.sort_values(by=['user_id'])
CustomerLoanVisitData.to_csv( "CombinedData.csv", index=False, encoding='utf-8-sig')
Message=["Done! Check you directory for the final file"]
Message


['Done! Check you directory for the final file']